## Steps to merge the global Free-flowing river and GRanD dam datasets
### Join datasets based on GOID
###  FFR dataset is very large - looped through a geometry filter based on continent boundaries to load FFR in chunks

### 6/23/21

In [32]:
# Import packages, set working directory
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import fiona
import geopandas as gpd
import rasterio as rio
import requests
import seaborn as sns
import zipfile
import earthpy as et
import urllib.request
import warnings

# Ignore runtime warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [33]:
# Set working directory
wrk_dir_path = os.path.join(et.io.HOME,"earth-analytics","data")

# Conditional statement to set working directory if it exists or create one if it doesn't exist
if os.path.exists(wrk_dir_path):
    os.chdir(wrk_dir_path)
    print("The specified directory exists and is now the working directory")
else:
    os.makedirs(wrk_dir_path)
    os.chdir(wrk_dir_path)
    print("There currently is not a directory", wrk_dir_path,
          "but it is being created and called to be the working directory.")

# DATA FILE PATHS
# Continent boundaries
continent_path = os.path.join("earthpy-downloads", "continent-poly", "Continents.shp")

# Country boundaries
countries_path = os.path.join("earthpy-downloads", "country-borders",
                              "99bfd9e7-bb42-4728-87b5-07f8c8ac631c2020328-1-1vef4ev.lu5nk.shp")

# Define path to GRanD file
dam_path = os.path.join("earthpy-downloads", "GRanD_Version_1_3","GRanD_dams_v1_3.shp")

# Define path to unzipped FFR
ffr_path = os.path.join("earthpy-downloads", "Mapping%20the%20worlds%20free-flowing%20rivers_Data_Geodatabase",
                          "Mapping the worlds free-flowing rivers_Data_Geodatabase",
                          "FFR_river_network.gdb")

# CONDITIONAL STATEMENTS TO SEE IF DATA ARE ALREADY STORED LOCALLY AND IF NOT, GET DATA
if os.path.exists(continent_path):
    continent_border = gpd.read_file(continent_path)
    print("The continent path exists and data are downloaded")
else:
    et.data.get_data(url="https://ndownloader.figshare.com/files/23392280")
    continent_border = gpd.read_file(continent_path)
    print("Needed to retrieve continent data from url and data are downloaded")
    
if os.path.exists(countries_path):
    countries_border = gpd.read_file(countries_path)
    print("The countries path exists and data are downloaded")
else:
    et.data.get_data(url="https://ndownloader.figshare.com/files/22507058")
    countries_border = gpd.read_file(countries_path)
    print("Needed to retrieve country data from url and data are downloaded")
    
if os.path.exists(dam_path):
    global_dams = gpd.read_file(dam_path)
    print("The dam path exists and data are downloaded")
else:
    et.data.get_data(url="https://ln.sync.com/dl/bd47eb6b0/anhxaikr-62pmrgtq-k44xf84f-pyz4atkm/view/default/447819520013")#
    with zipfile.ZipFile("GRanD_Version_1_3.zip","r") as zip_ref:
        zip_ref.extractall("earthpy-downloads")
    print('File is unzipped in earthpy-downloads folder') 
    global_dams = gpd.read_file(dam_path)
    print("Unzipped dam data are downloaded")

# Free-flowing rivers - just wanted to see if data are unzipped somewhere
if os.path.exists(river_path):
    print("The river path exists with unzipped data")
else:
    et.data.get_data(url="https://ndownloader.figshare.com/files/15090536")
    with zipfile.ZipFile("Mapping the worlds free-flowing rivers_Data_Geodatabase.zip","r") as zip_ref:
        zip_ref.extractall("earthpy-downloads")
    print('River file is now unzipped in earthpy-downloads folder')
    

The specified directory exists and is now the working directory
The continent path exists and data are downloaded
The countries path exists and data are downloaded
The dam path exists and data are downloaded
The river path exists with unzipped data


In [34]:
# Load processed GRanD.csv with FFR unique ID
grand_csv_path = os.path.join(
    "GRanDv13_GOID.csv")

# Import .csv into Pandas dataframe
grand_csv = pd.read_csv(grand_csv_path,
                        na_values=[-9999])

# See number of dams missing GOID
print('Number of observations without GOID =', grand_csv['GOID'].isnull().sum())

# Need to fill in NAs to be able to convert to an integer
grand_csv = grand_csv.fillna(-9999)

# REDUCE COLUMNS TO ONLY ONES OF INTEREST FOR MERGE
grand_csv = grand_csv[["GRAND_ID","GOID"]]

# Convert GOID to integer
grand_csv['GOID'] = grand_csv['GOID'].astype(int)

Number of observations without GOID = 7


In [35]:
# PROCESS GRanD shapefile dataframe
# Convert catchment area to float
global_dams['CATCH_SKM'] = global_dams['CATCH_SKM'].astype(float)

# Convert GRAND_ID to an object to merge with processed dam data subset
global_dams['GRAND_ID'] = global_dams['GRAND_ID'].astype(str)

In [36]:
# MERGE PROCESSED GRAND CSV WITH SHAPEFILE GRAND TO GET GOID COLUMN
grand_proc = global_dams.merge(grand_csv, on= "GRAND_ID", how= "inner")

grand_proc = grand_proc.drop(columns=['COUNTRY','geometry'])
grand_proc['GOID'] = grand_proc['GOID'].astype(int)

In [37]:
# FUNCTION TO CREATE EXPANDED HYDRO-ELECTRIC CLASSES
def reclass_dam(new_class):
    """ Create an expanded dam main use category that indicates whether a dam
    is listed as having hydroelectric usage but may have a different main
    purpose listed. This function creates new classes that combine the
    main use and secondary hydroelectric use dam category as new labels.
    
    Parameters
    -----------
    new_class : string
        Name of new dam main use column to be created
    
    Returns
    --------
    grand_proc :geodataframe
        The geodataframe with the added newly created column
    """
    grand_proc[new_class]=grand_proc['MAIN_RED']
    grand_proc.loc[(grand_proc['USE_ELEC'] =='Major') |
                        (grand_proc['USE_ELEC'] =='Sec')  & 
                        (grand_proc['MAIN_RED'] == 'Irrigation'), new_class] = 'Hydro_irrig'

    grand_proc.loc[(grand_proc['USE_ELEC'] =='Major') |
                        (grand_proc['USE_ELEC'] =='Sec')  & 
                        (grand_proc['MAIN_RED'] == 'Water supply'), new_class] = 'Hydro_water'

    grand_proc.loc[(grand_proc['USE_ELEC'] =='Major') |
                        (grand_proc['USE_ELEC'] =='Sec')  & 
                        (grand_proc['MAIN_RED'] == 'Navigation'), new_class] = 'Hydro_navig'

    grand_proc.loc[(grand_proc['USE_ELEC'] =='Major') |
                        (grand_proc['USE_ELEC'] =='Sec')  & 
                        (grand_proc['MAIN_RED'] == 'Flood control'), new_class] = 'Hydro_flood'

    grand_proc.loc[(grand_proc['USE_ELEC'] =='Major') |
                        (grand_proc['USE_ELEC'] =='Sec')  & 
                        (grand_proc['MAIN_RED'] == 'Other_expanded'), new_class] = 'Hydro_other'
    
    return grand_proc



# FUNCTION TO JOIN FFR AND GRAND DATA based on GOID

def ffr_grand_join(ffr_dat,
                  grand_dat):
    """ Merge Free-flowing rivers data subset with processed GRanD data and create
        new geodataframe of matching river segments and with dam observations based on GOID.
    
    Parameters
    -----------
    ffr_dat : geodataframe 
        Subset of free-flowing river data
    grand_dat : geodataframe
        Processed GRanD dam dataset with new columns and GOID added
    
    Returns
    ----------
    ffr_grand : geodataframe
        A geodataframe of matching FFR and GRanD observations that maintains the river geometry
    """

    ffr_grand = ffr_dat.merge(grand_dat, on="GOID", how="inner")
    
    return ffr_grand

In [38]:
# 1) MAIN_RED - reduce dams 'recreation','fisheries', & 'other' to a general 'other' category
grand_proc['MAIN_RED']=grand_proc['MAIN_USE']

grand_proc.loc[(grand_proc['MAIN_USE'] == 'Recreation') |
                         (grand_proc['MAIN_USE'] == 'Fisheries') |
                         (grand_proc['MAIN_USE'] == 'Other'), 'MAIN_RED'] = 'Other_expanded'

In [39]:
# 2) MAIN_HYDEXP - use function to reclassify dams with hydroelectricity listed as secondary use
reclass_dam(new_class='MAIN_HYDEXP')
grand_proc.columns
grand_proc['MAIN_HYDEXP'].value_counts()

Hydroelectricity    1822
Irrigation          1527
Water supply         789
Other_expanded       502
Flood control        412
Hydro_irrig          355
Hydro_flood          163
Hydro_water          100
Navigation            34
Hydro_other           33
Hydro_navig           22
Name: MAIN_HYDEXP, dtype: int64

In [40]:
# 3) MAIN_HYDRO - Reclassify dams with hydroelectricity listed as any use
grand_proc['MAIN_HYDRO']=grand_proc['MAIN_HYDEXP']
grand_proc.loc[(grand_proc['MAIN_HYDEXP']== 'Hydro_irrig')|
                (grand_proc['MAIN_HYDEXP']== 'Hydro_water')|
                (grand_proc['MAIN_HYDEXP']== 'Hydro_navig')|
                (grand_proc['MAIN_HYDEXP']== 'Hydro_flood')|
                (grand_proc['MAIN_HYDEXP']== 'Hydro_other')|
                (grand_proc['MAIN_HYDEXP']== 'Hydroelectricity'),'MAIN_HYDRO'] = 'Hydroelectricity'

# Check number of observations missing dam usage to make sure reclassification adds up
if grand_proc['MAIN_HYDEXP'].count()==global_dams['MAIN_USE'].count():
    print("Number of dams not missing main use is correct and equal to",global_dams['MAIN_USE'].count())
else:
    print("Number of dams not missing main use are different between datasets")

Number of dams not missing main use is correct and equal to 5759


In [41]:
# 4) CREATE DUMMY VARIABLE TO INDICIATE WHETHER HYDROELECTRICITY IS ANY PURPOSE (Y = 1; N = 0)
grand_proc['HYD_dummy'] = grand_proc['MAIN_HYDEXP'].apply(lambda x: '1' if (x=='Hydroelectricity')|(x=='Hydro_irrig')|
                                                 (x=='Hydro_flood')|
                                                 (x=='Hydro_water')|
                                                 (x=='Hydro_navig')|
                                                 (x=='Hydro_other') else '0')

grand_proc['HYD_dummy'].value_counts()

0    4825
1    2495
Name: HYD_dummy, dtype: int64

In [42]:
# MERGE FFR and Processed GRanD using merge function and 
#  looping through geometry filter based on continent boundary

# Create an empty list to populate with geodataframes
merge_dfs = []
for acontinent in continent_border['CONTINENT']:
    ffr_subset = gpd.read_file(ffr_path,
                              mask=continent_border[continent_border['CONTINENT']==
                                                    acontinent],)
    ffr_grand_subset = ffr_grand_join(ffr_dat = ffr_subset,
                                     grand_dat = grand_proc)
    merge_dfs.append(ffr_grand_subset)

all_ffr_grand = pd.concat(merge_dfs)

all_ffr_grand

,REACH_ID,GOID,NOID,NUOID,NDOID,CON_ID,CONTINENT,COUNTRY,BAS_ID,BAS_NAME,...,URL,QUALITY,EDITOR,LONG_DD,LAT_DD,POLY_SRC,MAIN_RED,MAIN_HYDEXP,MAIN_HYDRO,HYD_dummy
0,10000030.0,30,30,29_48,0,4,Africa,Tunisia,2612222,,...,None,4: Poor,McGill-BL,9.755616,37.167722,SWBD,None,None,None,0
1,10000047.0,47,47,49_61,19,4,Africa,Tunisia,2612222,,...,None,2: Good,McGill-BL,9.475369,37.179550,SWBD,Irrigation,Irrigation,Irrigation,0
2,10000117.0,117,117,111_123,0,4,Africa,Tunisia,2617710,,...,None,3: Fair,McGill-BL,8.939581,37.022558,SWBD,Irrigation,Irrigation,Irrigation,0
3,10000120.0,120,120,,72,4,Africa,Tunisia,2612222,,...,None,2: Good,McGill-BL,9.545999,37.058462,SWBD,Irrigation,Irrigation,Irrigation,0
4,10000125.0,125,125,,126,4,Africa,Tunisia,2628393,,...,None,3: Fair,McGill-BL,9.863779,37.045283,SWBD,Other_expanded,Other_expanded,Other_expanded,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1246,20680349.0,2207900,2207900,,2208802,3,Europe,Spain,2614762,Guadalhorce,...,None,3: Fair,McGill-BL,-3.496226,36.861264,McGill,Irrigation,Hydro_irrig,Hydroelectricity,1
1247,20681039.0,2208590,2208590,2207897_2208072,2209070,3,Europe,Spain,2619115,Guadalete,...,None,3: Fair,McGill-BL,-5.759233,36.793809,SWBD,Hydroelectricity,Hydroelectricity,Hydroelectricity,1
1248,20683299.0,2210850,2210850,,2210495,3,Europe,Spain,2619115,Guadalete,...,None,3: Fair,McGill-BL,-5.562922,36.662081,SWBD,Irrigation,Irrigation,Irrigation,0
1249,20687969.0,2215520,2215520,2215351_2215352,2216296,3,Europe,Spain,2640120,,...,None,3: Fair,McGill-BL,-5.739531,36.377117,SWBD,Irrigation,Irrigation,Irrigation,0


In [43]:
# EXPORT PROCESSED DATA
all_ffr_grand.dtypes
# WRITE CSV FILE OF MERGED FFR AND GRAND DATA FOR WORLD
all_ffr_grand.to_csv (r'C:\Users\Owner\earth-analytics\data\FFR_GRAND.csv', index = False, header=True)

# WRITE SHAPEFILE OF AFRICA RIVERS
# CREATE PATH FOR EXPORT
output_path = os.path.join(
    "FFR_GRAND_gdf","FFR_GRAND.shp")

all_ffr_grand.to_file(output_path)

<ipython-input-43-cca47f293fec>:11: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  all_ffr_grand.to_file(output_path)


In [45]:
# Data exploration to make sure it did what we expected
print(all_ffr_grand.shape) # 7303 out of 7320 Grand dams
print(grand_proc.shape)
print(all_ffr_grand['MAIN_USE'].count())
print(grand_proc['MAIN_USE'].count())
print(all_ffr_grand['CONTINENT'].value_counts())

(7303, 100)
(7320, 62)
5748
5759
North America    2125
Asia             2100
Europe           1524
Africa            759
South America     507
Australia         288
Name: CONTINENT, dtype: int64
